In [1]:
import collections
from pathlib import Path

import numpy as np
import torch
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
import torch.nn as nn

from utils import load_obj_tsv
from dataset.vqa import VQADataset,VQATorchDataset, VQAEvaluator

from models.lxrt_adaptive import Model_Args

I0125 23:37:24.470238 140117533944704 file_utils.py:35] PyTorch version 1.4.0+cpu available.
/glob/intel-python/python3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
home = str(Path.home())
MSCOCO_IMGFEAT_ROOT = home + '/data/mscoco_imgfeat/'
VQA_DATA_ROOT = home+'/data/vqa/'
load_lxmert_qa_path = home+'/snap/pretrained/model'

In [3]:
class Args():
    def __init__(self):
        self.tiny = True
        self.bs = 1

In [4]:
args = Args()

In [5]:
DataTuple = collections.namedtuple("DataTuple", 'dataset loader evaluator')

def get_data_tuple(path: str, mscoco_path: str, splits: str, tiny: bool,bs:int, shuffle=False, drop_last=False) -> DataTuple:
    dset = VQADataset(path,splits)
    tset = VQATorchDataset(dset,mscoco_path,tiny)
    evaluator = VQAEvaluator(dset)
    pin_memory = True if torch.cuda.is_available() else False
    data_loader = DataLoader(
        tset, batch_size=bs,
        shuffle=shuffle, num_workers=1,
        drop_last=drop_last, pin_memory=pin_memory
    )

    return DataTuple(dataset=dset, loader=data_loader, evaluator=evaluator)

train_tuple = get_data_tuple(VQA_DATA_ROOT, MSCOCO_IMGFEAT_ROOT, 'train,nominival', args.tiny, args.bs,True,True)
valid_tuple = get_data_tuple(VQA_DATA_ROOT, MSCOCO_IMGFEAT_ROOT,'minival',args.tiny,args.bs,True,True)

0it [00:00, ?it/s]

Load 632117 data from split(s) train,nominival.
Start to load Faster-RCNN detected objects from /home/u37216/data/mscoco_imgfeat/train2014_obj36.tsv


494it [00:02, 180.98it/s]
17it [00:00, 169.44it/s]

Loaded 500 images in file /home/u37216/data/mscoco_imgfeat/train2014_obj36.tsv in 2 seconds.
Start to load Faster-RCNN detected objects from /home/u37216/data/mscoco_imgfeat/val2014_obj36.tsv


492it [00:02, 179.85it/s]


Loaded 500 images in file /home/u37216/data/mscoco_imgfeat/val2014_obj36.tsv in 2 seconds.
Use 2810 data in torch dataset



17it [00:00, 162.15it/s]

Load 25994 data from split(s) minival.
Start to load Faster-RCNN detected objects from /home/u37216/data/mscoco_imgfeat/val2014_obj36.tsv


490it [00:02, 177.66it/s]

Loaded 500 images in file /home/u37216/data/mscoco_imgfeat/val2014_obj36.tsv in 2 seconds.
Use 2546 data in torch dataset



In [21]:
from models.lxrt_adaptive import VQAModel_Adaptive
#from tasks.vqa_model import VQAModel

In [22]:
model_args = Model_Args(9,6,6)
model_args.sparse = False #args.sparse

In [23]:
adapt_span_params = {'adapt_span_enabled': True, 'attn_span': 1024, 'adapt_span_loss_coeff': 0.000005,
                     'adapt_span_ramp': 32, 'adapt_span_init': 0.6, 'adapt_span_cache': True, 
                     'nb_heads': 12,'bs': args.bs, 'mask_size': [20,36]}

In [24]:
model = VQAModel_Adaptive(train_tuple[0].num_answers,model_args,adapt_span_params)
#model = VQAModel(train_tuple[0].num_answers,model_args)

I0125 23:50:06.241447 140117533944704 tokenization_utils.py:398] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/u37216/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
I0125 23:50:06.594721 140117533944704 modeling.py:776] loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /home/u37216/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
I0125 23:50:06.600104 140117533944704 modeling.py:784] extracting archive file /home/u37216/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /home/u37216/tmp/tmp9y19t38t
I0125 23:50:12.227200 14011753394470

LXRT encoder with 9 l_layers, 6 x_layers, and 6 r_layers.
Using adaptive variant


In [11]:
ques_id,feats,boxes,sent,target = next(iter(train_tuple[1]))

Traceback (most recent call last):
  File "/glob/intel-python/python3/lib/python3.6/multiprocessing/util.py", line 262, in _run_finalizers
    finalizer()
  File "/glob/intel-python/python3/lib/python3.6/multiprocessing/util.py", line 186, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/glob/intel-python/python3/lib/python3.6/shutil.py", line 480, in rmtree
    _rmtree_safe_fd(fd, path, onerror)
  File "/glob/intel-python/python3/lib/python3.6/shutil.py", line 438, in _rmtree_safe_fd
    onerror(os.unlink, fullname, sys.exc_info())
  File "/glob/intel-python/python3/lib/python3.6/shutil.py", line 436, in _rmtree_safe_fd
    os.unlink(name, dir_fd=topfd)
OSError: [Errno 16] Device or resource busy: '.nfs00000018801e4e11000006fb'


In [27]:
# logit = model(feats,boxes,sent)

In [13]:
from thop import profile

In [25]:
macs, params = profile(model, inputs=(feats,boxes,sent))

--- Logging error ---
Traceback (most recent call last):
  File "/glob/intel-python/python3/lib/python3.6/logging/__init__.py", line 992, in emit
    msg = self.format(record)
  File "/glob/intel-python/python3/lib/python3.6/logging/__init__.py", line 838, in format
    return fmt.format(record)
  File "/glob/intel-python/python3/lib/python3.6/site-packages/absl/logging/__init__.py", line 928, in format
    return prefix + super(PythonFormatter, self).format(record)
  File "/glob/intel-python/python3/lib/python3.6/logging/__init__.py", line 575, in format
    record.message = record.getMessage()
  File "/glob/intel-python/python3/lib/python3.6/logging/__init__.py", line 338, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/glob/intel-python/python3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/glob/intel-python/python3/lib/python3.6/runpy.py", line 85, in _run_code

I0125 23:50:20.580325 140117533944704 profile.py:90] Register FLOP counter for module Linear(in_features=2048, out_features=768, bias=True)
--- Logging error ---
Traceback (most recent call last):
  File "/glob/intel-python/python3/lib/python3.6/logging/__init__.py", line 992, in emit
    msg = self.format(record)
  File "/glob/intel-python/python3/lib/python3.6/logging/__init__.py", line 838, in format
    return fmt.format(record)
  File "/glob/intel-python/python3/lib/python3.6/site-packages/absl/logging/__init__.py", line 928, in format
    return prefix + super(PythonFormatter, self).format(record)
  File "/glob/intel-python/python3/lib/python3.6/logging/__init__.py", line 575, in format
    record.message = record.getMessage()
  File "/glob/intel-python/python3/lib/python3.6/logging/__init__.py", line 338, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/glob/intel-python/python3/lib/python3.6/runpy.py", 

I0125 23:50:20.602679 140117533944704 profile.py:90] Register FLOP counter for module Linear(in_features=768, out_features=768, bias=True)
I0125 23:50:20.603977 140117533944704 profile.py:90] Register FLOP counter for module Linear(in_features=768, out_features=768, bias=True)
I0125 23:50:20.605384 140117533944704 profile.py:90] Register FLOP counter for module Linear(in_features=768, out_features=768, bias=True)
I0125 23:50:20.606577 140117533944704 profile.py:90] Register FLOP counter for module Dropout(p=0.1, inplace=False)
--- Logging error ---
Traceback (most recent call last):
  File "/glob/intel-python/python3/lib/python3.6/logging/__init__.py", line 992, in emit
    msg = self.format(record)
  File "/glob/intel-python/python3/lib/python3.6/logging/__init__.py", line 838, in format
    return fmt.format(record)
  File "/glob/intel-python/python3/lib/python3.6/site-packages/absl/logging/__init__.py", line 928, in format
    return prefix + super(PythonFormatter, self).format(reco

I0125 23:50:20.630303 140117533944704 profile.py:90] Register FLOP counter for module Linear(in_features=768, out_features=3072, bias=True)
--- Logging error ---
Traceback (most recent call last):
  File "/glob/intel-python/python3/lib/python3.6/logging/__init__.py", line 992, in emit
    msg = self.format(record)
  File "/glob/intel-python/python3/lib/python3.6/logging/__init__.py", line 838, in format
    return fmt.format(record)
  File "/glob/intel-python/python3/lib/python3.6/site-packages/absl/logging/__init__.py", line 928, in format
    return prefix + super(PythonFormatter, self).format(record)
  File "/glob/intel-python/python3/lib/python3.6/logging/__init__.py", line 575, in format
    record.message = record.getMessage()
  File "/glob/intel-python/python3/lib/python3.6/logging/__init__.py", line 338, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/glob/intel-python/python3/lib/python3.6/runpy.py", 

I0125 23:50:20.650559 140117533944704 profile.py:90] Register FLOP counter for module Linear(in_features=768, out_features=768, bias=True)
I0125 23:50:20.651755 140117533944704 profile.py:90] Register FLOP counter for module Linear(in_features=768, out_features=768, bias=True)
I0125 23:50:20.652947 140117533944704 profile.py:90] Register FLOP counter for module Linear(in_features=768, out_features=768, bias=True)
I0125 23:50:20.653977 140117533944704 profile.py:90] Register FLOP counter for module Dropout(p=0.1, inplace=False)
--- Logging error ---
Traceback (most recent call last):
  File "/glob/intel-python/python3/lib/python3.6/logging/__init__.py", line 992, in emit
    msg = self.format(record)
  File "/glob/intel-python/python3/lib/python3.6/logging/__init__.py", line 838, in format
    return fmt.format(record)
  File "/glob/intel-python/python3/lib/python3.6/site-packages/absl/logging/__init__.py", line 928, in format
    return prefix + super(PythonFormatter, self).format(reco

...

I0125 23:50:21.550502 140117533944704 profile.py:90] Register FLOP counter for module Linear(in_features=768, out_features=3072, bias=True)
--- Logging error ---
Traceback (most recent call last):
  File "/glob/intel-python/python3/lib/python3.6/logging/__init__.py", line 992, in emit
    msg = self.format(record)
  File "/glob/intel-python/python3/lib/python3.6/logging/__init__.py", line 838, in format
    return fmt.format(record)
  File "/glob/intel-python/python3/lib/python3.6/site-packages/absl/logging/__init__.py", line 928, in format
    return prefix + super(PythonFormatter, self).format(record)
  File "/glob/intel-python/python3/lib/python3.6/logging/__init__.py", line 575, in format
    record.message = record.getMessage()
  File "/glob/intel-python/python3/lib/python3.6/logging/__init__.py", line 338, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/glob/intel-python/python3/lib/python3.6/runpy.py", 

  File "/glob/intel-python/python3/lib/python3.6/site-packages/tornado/stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "/glob/intel-python/python3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/glob/intel-python/python3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 235, in dispatch_shell
    handler(stream, idents, msg)
  File "/glob/intel-python/python3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 399, in execute_request
    user_expressions, allow_stdin)
  File "/glob/intel-python/python3/lib/python3.6/site-packages/ipykernel/ipkernel.py", line 196, in do_execute
    res = shell.run_cell(code, store_history=store_history, silent=silent)
  File "/glob/intel-python/python3/lib/python3.6/site-packages/ipykernel/zmqshell.py", line 533, in run_cell
    return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)
  File "/glob/intel-python/p

  File "/glob/intel-python/python3/lib/python3.6/logging/__init__.py", line 1442, in _log
    self.handle(record)
  File "/glob/intel-python/python3/lib/python3.6/logging/__init__.py", line 1452, in handle
    self.callHandlers(record)
  File "/glob/intel-python/python3/lib/python3.6/logging/__init__.py", line 1514, in callHandlers
    hdlr.handle(record)
  File "/glob/intel-python/python3/lib/python3.6/site-packages/absl/logging/__init__.py", line 891, in handle
    return self._current_handler.handle(record)
  File "/glob/intel-python/python3/lib/python3.6/logging/__init__.py", line 863, in handle
    self.emit(record)
  File "/glob/intel-python/python3/lib/python3.6/site-packages/absl/logging/__init__.py", line 825, in emit
    self._log_to_stderr(record)
  File "/glob/intel-python/python3/lib/python3.6/site-packages/absl/logging/__init__.py", line 797, in _log_to_stderr
    super(PythonHandler, self).emit(record)
Message: 'THOP has not implemented counting method for '
Arguments: (

  File "/glob/intel-python/python3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/glob/intel-python/python3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/glob/intel-python/python3/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/glob/intel-python/python3/lib/python3.6/site-packages/tornado/ioloop.py", line 888, in start
    handler_func(fd_obj, events)
  File "/glob/intel-python/python3/lib/python3.6/site-packages/tornado/stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "/glob/intel-python/python3/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py", line 440, in _handle_events
    self._handle_recv()
  File "/glob/intel-python/python3/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py", line 472, in _handle_recv
    self._run_callback(call

  File "/glob/intel-python/python3/lib/python3.6/site-packages/absl/logging/__init__.py", line 825, in emit
    self._log_to_stderr(record)
  File "/glob/intel-python/python3/lib/python3.6/site-packages/absl/logging/__init__.py", line 797, in _log_to_stderr
    super(PythonHandler, self).emit(record)
Message: 'THOP has not implemented counting method for '
Arguments: (GeLU(),)
--- Logging error ---
Traceback (most recent call last):
  File "/glob/intel-python/python3/lib/python3.6/logging/__init__.py", line 992, in emit
    msg = self.format(record)
  File "/glob/intel-python/python3/lib/python3.6/logging/__init__.py", line 838, in format
    return fmt.format(record)
  File "/glob/intel-python/python3/lib/python3.6/site-packages/absl/logging/__init__.py", line 928, in format
    return prefix + super(PythonFormatter, self).format(record)
  File "/glob/intel-python/python3/lib/python3.6/logging/__init__.py", line 575, in format
    record.message = record.getMessage()
  File "/glob/int

### macs, params (Default)
(12074093568.0, 237557824.0)

In [26]:
macs,params

(12074093568.0, 237558320.0)

In [15]:
from thop import clever_format
macs, params = clever_format([macs, params], "%.3f")

In [16]:
macs

'12.074G'

In [17]:
params

'237.558M'

## Adaptive
macs: 12.074G
params: 237.558M

## Default
macs: 12.074G
params: 237.558M